<div style="display: flex; justify-content: space-between; align-items: center;">
    <div style="text-align: left; flex: 4;">
        <strong>Author:</strong> Amirhossein Heydari — 
        📧 <a href="mailto:amirhosseinheydari78@gmail.com">amirhosseinheydari78@gmail.com</a> — 
        🐙 <a href="https://github.com/mr-pylin/media-processing-workshop" target="_blank" rel="noopener">github.com/mr-pylin</a>
    </div>
    <div style="display: flex; justify-content: flex-end; flex: 1; gap: 8px; align-items: center; padding: 0;">
        <a href="https://opencv.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/opencv/logo/OpenCV_logo_no_text-1.svg"
                 alt="OpenCV Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://pillow.readthedocs.io/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/pillow/logo/pillow-logo-248x250.png"
                 alt="PIL Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scikit-image.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/scikit-image/logo/logo.png"
                 alt="scikit-image Logo"
                 style="max-height: 48px; width: auto;">
        </a>
        <a href="https://scipy.org/" target="_blank" rel="noopener noreferrer">
            <img src="../../assets/images/libraries/scipy/logo/logo.svg"
                 alt="SciPy Logo"
                 style="max-height: 48px; width: auto;">
        </a>
    </div>
</div>
<hr>


**Table of contents**<a id='toc0_'></a>    
- [Color Theory Fundamentals](#toc1_)    
  - [Human Vision Basics](#toc1_1_)    
  - [Human Color Gamut](#toc1_2_)    
- [Dependencies](#toc2_)    
- [Load Images](#toc3_)    
- [Color Space Conversion](#toc4_)    
  - [RGB - Grayscale](#toc4_1_)    
    - [Manual](#toc4_1_1_)    
    - [Using OpenCV](#toc4_1_2_)    
    - [Using PIL](#toc4_1_3_)    
    - [Using Scikit-Image](#toc4_1_4_)    
  - [RGB - BGR](#toc4_2_)    
    - [Manual](#toc4_2_1_)    
    - [Using OpenCV](#toc4_2_2_)    
  - [RGB - YUV](#toc4_3_)    
    - [Manual](#toc4_3_1_)    
    - [Using OpenCV](#toc4_3_2_)    
  - [RGB - YCbCr (YCC)](#toc4_4_)    
    - [Manual](#toc4_4_1_)    
    - [Using OpenCV](#toc4_4_2_)    
  - [RGB - HSV](#toc4_5_)    
    - [Using OpenCV](#toc4_5_1_)    
  - [RGB - CMYK](#toc4_6_)    
    - [Using PIL](#toc4_6_1_)    
- [Visualize using Custom Color Maps](#toc5_)    
  - [Indexed Image](#toc5_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Color Theory Fundamentals](#toc0_)


## <a id='toc1_1_'></a>[Human Vision Basics](#toc0_)

- **Spectral range**: 380-700nm  
- **Color discrimination**: ~10-11 million colors (optimal conditions)  
- **Cone types**: S (420nm), M (530nm), L (560nm) peak sensitivity

<div style="display: flex; justify-content: center; gap: 40px; margin: 20px 0;">
  <div style="flex: 1; text-align: center; padding: 10px;">
    <img src="../../assets/images/third_party/Cone-fundamentals-with-srgb-spectrum.svg"
         alt="Cone-fundamentals-with-srgb-spectrum.svg"
         style="max-height: 250px; object-fit: contain;">
    <div style="margin-top: 8px;">Normalized responsivity spectra of human cone cells, S, M, and L types</div>
  </div>
  <div style="flex: 1; text-align: center; padding: 10px;">
    <img src="../../assets/images/third_party/ConeMosaics.png"
         alt="ConeMosaics.png"
         style="max-height: 250px; object-fit: contain;">
    <div style="margin-top: 8px;">Distribution of cone cells in the fovea of an individual with normal color vision (left), and a color blind (protanopic) retina</div>
  </div>
</div>

<div style="text-align: center; margin-top: 15px;">
  Visual representation of cone cell characteristics and distribution (©️ 
  <a href="https://en.wikipedia.org/wiki/Cone_cell">Wikipedia - Cone cell</a>)
</div>


## <a id='toc1_2_'></a>[Human Color Gamut](#toc0_)

The **human color gamut** represents all colors perceivable by human vision, visualized through the CIE 1931 chromaticity diagram.  
Approximately ~8% of males and ~0.5% of females are affected by color blindness, which impacts their perception of this gamut.

<figure style="text-align: center; margin: 0;">
  <div style="display: flex; justify-content: center; gap: 20px;">
    <div style="flex: 1; background-color: lightgray; padding: 10px; display: flex; align-items: center; justify-content: center;">
      <img src="../../assets/images/third_party/Rechteckspektrum_sRGB.svg"
           alt="Rechteckspektrum_sRGB.svg"
           style="max-width: 100%; height: auto;">
    </div>
    <div style="flex: 1; background-color: lightgray; padding: 10px; display: flex; align-items: center; justify-content: center;">
      <img src="../../assets/images/third_party/Cie_Chart_with_sRGB_gamut_by_spigget.png"
           alt="Cie_Chart_with_sRGB_gamut_by_spigget.png"
           style="max-width: 80%; height: auto;">
    </div>
    <div style="flex: 1; background-color: lightgray; padding: 10px; display: flex; align-items: center; justify-content: center;">
      <img src="../../assets/images/third_party/CIE1931xy_gamut_comparison.svg"
           alt="CIE1931xy_gamut_comparison.svg"
           style="max-width: 90%; height: auto;">
    </div>
  </div>
  <figcaption>
    Visual representations of color gamuts in sRGB space (©️ 
    <a href="https://en.wikipedia.org/wiki/Gamut">Wikipedia - Gamut</a>)
  </figcaption>
</figure>

**CIE Coordinates (Simplified DIP Approach):**

In digital image processing, chromaticity coordinates can be simplified as:

- **x**: Red component
- **y**: Green component  
- **z**: Blue component (z = 1-x-y)

**Example**: 780nm wavelength → x=0.74, y=0.26, z=0.00 (deep red)

**RGB Display Limitations:**

Modern displays reproduce only a subset of human-visible colors:

- **sRGB**: ~35% coverage
- **Adobe RGB**: ~50% coverage
- **DCI-P3**: ~45% coverage

**2D vs 3D Color Space:**

- **2D diagram**: Chromaticity (hue/saturation) at constant luminance
- **3D space**: Adds brightness dimension (Y-value)


# <a id='toc2_'></a>[Dependencies](#toc0_)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
from matplotlib.colors import ListedColormap
from numpy.typing import NDArray
from PIL import Image

In [ ]:
np.set_printoptions(linewidth=120)

# <a id='toc3_'></a>[Load Images](#toc0_)

In [ ]:
image_1 = Image.open(fp="../../assets/images/dip_3rd/CH06_Fig0638(a)(lenna_RGB).tif")
image_2 = Image.open(fp="../../assets/images/misc/lenna_rgba_indexed.png")

# PIL.Image.Image to np.ndarray
im_1 = np.array(image_1)
im_2 = np.array(image_2)

# plot
fig, axs = plt.subplots(1, 2, figsize=(8, 4), layout="compressed")
axs[0].imshow(image_1, vmin=0, vmax=255)
axs[0].set_title("RGB")
axs[0].axis("off")
axs[1].imshow(image_2, vmin=0, vmax=255)
axs[1].set_title("Indexed RGBA")
axs[1].axis("off")
plt.show()

In [ ]:
# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["Original", "Red Channel", "Green Channel", "Blue Channel"]
images = [im_1, im_1[:, :, 0], im_1[:, :, 1], im_1[:, :, 2]]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if len(img.shape) == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

# <a id='toc4_'></a>[Color Space Conversion](#toc0_)

📝 **Docs**:

- `cv2.cvtColor`: [docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html#gaf86c09fe702ed037c03c2bc603ceab14](https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html#gaf86c09fe702ed037c03c2bc603ceab14)
- Color Space Conversions [`cv2`]: [docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html](https://docs.opencv.org/master/d8/d01/group__imgproc__color__conversions.html)
- Modes [`PIL`]: [pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes](https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-modes)
- Colormap reference [`matplotlib`]: [matplotlib.org/stable/gallery/color/colormap_reference.html](https://matplotlib.org/stable/gallery/color/colormap_reference.html)
- ITU-R Recommendation BT.601-7 [an international technical standard]: [itu.int/dms_pubrec/itu-r/rec/bt/r-rec-bt.601-7-201103-i!!pdf-e.pdf](http://itu.int/dms_pubrec/itu-r/rec/bt/r-rec-bt.601-7-201103-i!!pdf-e.pdf)

## <a id='toc4_1_'></a>[RGB - Grayscale](#toc0_)

Converting an RGB image to grayscale simplifies the color information by reducing colors into shades of gray.   
This process serves several important purposes:

- **Data Reduction:**  
  Grayscale images require less memory and computational power than full-color images since they contain only one channel instead of three.

- **Preprocessing for Computer Vision:**  
  Many image processing and computer vision algorithms (e.g., edge detection, segmentation, pattern recognition) operate more efficiently or exclusively on grayscale images, as color information may be redundant or irrelevant.

- **Highlighting Intensity Information:**  
  Grayscale focuses solely on luminance or brightness variations, which often correspond better to structural details in images compared to color information.

- **Simplifying Analysis:**  
  In fields like medical imaging, document scanning, or texture analysis, grayscale images help isolate texture, shape, and contrast without the distraction of color.

🔢 **RGB to Grayscale Conversion Methods:**

1. **Luminance (perceptual weights, standard method):**  
   Human vision is more sensitive to green than to red and blue. The weighted sum is used in most image processing libraries:

   $$
   Y = 0.299 \, R + 0.587 \, G + 0.114 \, B
   $$

   ✅ **Use when:** Accurate brightness representation is needed, e.g., for human viewing, compression, or perceptual tasks.

2. **Averaging Method:**  
   Simply take the arithmetic mean of the three channels:

   $$
   Y = \frac{R + G + B}{3}
   $$

   ✅ **Use when:** Simplicity is more important than perceptual accuracy, e.g., quick visualization, basic preprocessing.

3. **L2 Norm (Euclidean norm method):**  
   Computes the magnitude of the RGB vector:

   $$
   Y = \sqrt{R^2 + G^2 + B^2 \over 3}
   $$

   ✅ **Use when:** Emphasizing overall intensity magnitude or when treating RGB as a vector in feature extraction or scientific imaging.

4. **Max/Min Channel Method:**  
   Take the maximum or minimum of R, G, B as the grayscale value:

   $$
   Y = \max(R, G, B) \quad \text{or} \quad Y = \min(R, G, B)
   $$

   ✅ **Use when:** Highlighting the strongest or weakest channel for edge detection or contrast emphasis.


### <a id='toc4_1_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_GS = np.array(
    [
        [0.299, 0.587, 0.114],
    ],
    dtype=np.float32,
)

# approximation using pseudo-inverse [best linear least-squares approximation]
GS_TO_RGB = np.linalg.pinv(RGB_TO_GS)

In [ ]:
def rgb_to_gs(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_GS.T).reshape(image.shape[:2])
    return np.clip(im, 0, 255).astype(np.uint8)


def gs_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    rgb = (image.reshape(-1, 1) @ GS_TO_RGB.T).reshape(*image.shape, 3)
    return np.clip(rgb, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_gs_1  = rgb_to_gs(im_1)
gs_1_to_rgb_1 = gs_to_rgb(im_1_to_gs_1)

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_1, gs_1_to_rgb_1]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc4_1_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_gs_2 = cv2.cvtColor(im_1, cv2.COLOR_RGB2GRAY)

# replicates the grayscale channel into all three RGB channels (still grayish image)
gs_2_to_rgb_2 = cv2.cvtColor(im_1_to_gs_2, cv2.COLOR_GRAY2RGB)

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_2, gs_2_to_rgb_2]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc4_1_3_'></a>[Using PIL](#toc0_)


In [ ]:
im_1_to_gs_3 = Image.fromarray(im_1).convert("L")
gs_3_to_rgb_3 = im_1_to_gs_3.convert("RGB")

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_3, gs_3_to_rgb_3]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc4_1_4_'></a>[Using Scikit-Image](#toc0_)


In [ ]:
im_1_to_gs_4 = ski.color.rgb2gray(im_1)
gs_4_to_rgb_4 = ski.color.gray2rgb(im_1_to_gs_4)

# plot
fig, axs = plt.subplots(1, 3, figsize=(12, 4), layout="compressed")
titles = ["Original", "RGB to GS", "GS to RGB (data loss)"]
images = [im_1, im_1_to_gs_4, gs_4_to_rgb_4]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc4_2_'></a>[RGB - BGR](#toc0_)

OpenCV uses the BGR (Blue, Green, Red) channel order as its default color format instead of the more common RGB order for historical and compatibility reasons:

- **Historical Legacy:**  
  Early versions of OpenCV were designed to be compatible with the Windows bitmap (BMP) image format, which stores pixel data in BGR order. Using BGR natively simplified reading and writing BMP files without additional channel reordering.

- **Performance Considerations:**  
  By matching the underlying image formats and memory layouts commonly used on Windows systems, OpenCV avoids unnecessary data copying or channel swapping, which can improve performance.

- **Consistency in the OpenCV Ecosystem:**  
  Over time, many OpenCV functions, tutorials, and sample codes have been built around BGR images. Changing the default now would break backward compatibility and require widespread refactoring.

- **Interoperability:**  
  Since OpenCV often interfaces with other libraries and legacy code that expect BGR ordering, it maintains BGR as a practical default.

🔢 **RGB to BGR Conversion (Channel Swap):**

$$
\begin{bmatrix}
B \\ G \\ R
\end{bmatrix}
=
\begin{bmatrix}
0 & 0 & 1 \\
0 & 1 & 0 \\
1 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
R \\ G \\ B
\end{bmatrix}
$$


### <a id='toc4_2_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_BGR = np.array(
    [
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ],
    dtype=np.float32,
)

BGR_TO_RGB = np.linalg.inv(RGB_TO_BGR)

In [ ]:
def rgb_to_bgr(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_BGR.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def bgr_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ BGR_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_bgr = rgb_to_bgr(im_1)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["BGR Image", "Red Channel", "Green Channel", "Blue Channel"]
images = [im_1_to_bgr] + [im_1_to_bgr[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc4_2_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_bgr = cv2.cvtColor(im_1, cv2.COLOR_RGB2BGR)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["BGR Image", "Blue Channel", "Green Channel", "Red Channel"]
images = [im_1_to_bgr] + [im_1_to_bgr[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc4_3_'></a>[RGB - YUV](#toc0_)

A color space designed to separate luminance (brightness) information from chrominance (color) information.  

- **Y** represents the luminance component (brightness).  
- **U** and **V** represent the chrominance components (color information).

**Purpose of RGB to YUV Conversion:**

- **Bandwidth Optimization in Video:**  
  Since the human eye is more sensitive to brightness details than color details, YUV allows chrominance (U, V) to be subsampled, reducing the amount of color data transmitted or stored without a significant loss in perceived quality.

- **Compression:**  
  Many video and image compression standards (like MPEG, JPEG, and H.264) use YUV because it enables better compression efficiency by treating luminance and chrominance separately.

- **Broadcast and Transmission:**  
  YUV was originally developed for analog TV broadcasting to maintain backward compatibility with black-and-white televisions (which only needed the luminance signal).

- **Color Manipulation:**  
  Separating luminance and chrominance can simplify tasks like brightness adjustments and color filtering.

🔢 **RGB to YUV Conversion:**

$$
\begin{bmatrix}
Y \\
U \\
V
\end{bmatrix}
=
\begin{bmatrix}
0.299 & 0.587 & 0.114 \\
-0.14713 & -0.28886 & 0.436 \\
0.615 & -0.51499 & -0.10001
\end{bmatrix}
\begin{bmatrix}
R \\
G \\
B
\end{bmatrix}
$$


### <a id='toc4_3_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_YUV = np.array(
    [
        [0.29900, 0.58700, 0.11400],
        [-0.14713, -0.28886, 0.43600],
        [0.61500, -0.51499, -0.10001],
    ],
    dtype=np.float32,
)

YUV_TO_RGB = np.linalg.inv(RGB_TO_YUV)
YUV_OFFSET = np.array([0, 128, 128])

In [ ]:
def rgb_to_yuv(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_YUV.T + YUV_OFFSET).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def yuv_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = ((image.reshape(-1, 3) - YUV_OFFSET) @ YUV_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_yuv = rgb_to_yuv(im_1)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YUV Image", "Y Channel", "U Channel", "V Channel"]
images = [im_1_to_yuv] + [im_1_to_yuv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc4_3_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_yuv = cv2.cvtColor(im_1, cv2.COLOR_RGB2YUV)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YUV Image", "Y Channel", "U Channel", "V Channel"]
images = [im_1_to_yuv] + [im_1_to_yuv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc4_4_'></a>[RGB - YCbCr (YCC)](#toc0_)

A color space used primarily in **digital video and image compression** (e.g., JPEG, MPEG, H.264), separating image luminance from chrominance components.

- **Y** represents the **luma** component (brightness approximation).
- **Cr** and **Cb** represent the **chrominance** components:
  - **Cb** is the blue-difference chroma component.
  - **Cr** is the red-difference chroma component.

**Purpose of RGB to Y'CrCb Conversion:**

- **Digital Compression Efficiency:**  
  Y'CrCb allows for **chroma subsampling** (like 4:2:0), significantly reducing data with minimal perceptual loss due to human visual system sensitivity.

- **Standard in Digital Media:**  
  Widely used in standards like **JPEG**, **MPEG**, and **broadcast video**, where it allows efficient encoding and transmission.

- **Compatibility with Color TV Signals:**  
  While derived from analog YUV, YCrCb is tailored for **digital representation**, including defined value ranges and offsets.

🔢 **RGB to Y'CrCb Conversion (BT.601 Standard):**

$$
\begin{bmatrix}
Y \\
Cb \\
Cr
\end{bmatrix}
=
\begin{bmatrix}
0.299 & 0.587 & 0.114 \\
-0.168736 & -0.331264 & 0.5 \\
0.5 & -0.418688 & -0.081312
\end{bmatrix}
\begin{bmatrix}
R \\
G \\
B
\end{bmatrix}
+
\begin{bmatrix}
0 \\
128 \\
128
\end{bmatrix}
$$

**✍️ Notes:**

There are multiple YCrCb (YCbCr) variants depending on usage:

- **Full-range YCbCr (e.g., JPEG):**
  - Y, Cb, and Cr all span the full 0–255 range.
  - Common in image formats like **JPEG** and **PNG**.

- **Limited-range YCbCr (e.g., BT.601/BT.709):**
  - Y is in the range **[16, 235]**, Cb/Cr in **[16, 240]**.
  - Used in **broadcast video**, **MPEG**, **H.264**, etc.


### <a id='toc4_4_1_'></a>[Manual](#toc0_)


In [ ]:
RGB_TO_YCBCR = np.array(
    [
        [0.299, 0.587, 0.114],
        [-0.168736, -0.331264, 0.5],
        [0.5, -0.418688, -0.081312],
    ],
    dtype=np.float32,
)

YCBCR_TO_RGB = np.linalg.inv(RGB_TO_YCBCR)
YCBCR_OFFSET = np.array([0, 128, 128])

In [ ]:
def rgb_to_ycbcr(image: NDArray) -> NDArray[np.uint8]:
    im = (image.reshape(-1, 3) @ RGB_TO_YCBCR.T + YCBCR_OFFSET).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)


def ycbcr_to_rgb(image: NDArray) -> NDArray[np.uint8]:
    im = ((image.reshape(-1, 3) - YCBCR_OFFSET) @ YCBCR_TO_RGB.T).reshape(image.shape)
    return np.clip(im, 0, 255).astype(np.uint8)

In [ ]:
im_1_to_ycrcb = rgb_to_ycbcr(im_1)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YCbCr Image", "Y Channel", "Cb Channel", "Cr Channel"]
images = [im_1_to_ycrcb] + [im_1_to_ycrcb[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

### <a id='toc4_4_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_ycrcb = cv2.cvtColor(im_1, cv2.COLOR_RGB2YCrCb)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["YCbCr Image", "Y Channel", "Cb Channel", "Cr Channel"]
images = [im_1_to_ycrcb] + [im_1_to_ycrcb[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc4_5_'></a>[RGB - HSV](#toc0_)

HSV stands for **Hue**, **Saturation**, and **Value** — a cylindrical color space that separates image intensity (value) from color information (hue and saturation).  
It is widely used in image processing, color pickers, and computer vision tasks.

- **Hue (H):**  
  Represents the color type (e.g., red, green, blue) as an angle in degrees on the color wheel.  
  Range is typically [0°, 360°], but often scaled to [0, 1] or [0, 255] in digital systems.

- **Saturation (S):**  
  Measures the vibrancy or purity of the color (0 = grayscale, 1 = full color). Range: [0, 1] or [0, 255].

- **Value (V):**  
  Indicates brightness, where 0 is black and 1 is the brightest. Range: [0, 1] or [0, 255].

**Purpose of RGB to HSV Conversion:**

- **Color Filtering and Detection:**  
  HSV makes it easier to isolate colors because hue is decoupled from brightness.

- **User Interfaces:**  
  HSV is more intuitive for human interpretation and interaction, especially in tools like color pickers.

- **Image Editing and Enhancement:**  
  Brightness or saturation can be adjusted independently of hue, simplifying color manipulation.

🔢 **RGB to HSV Conversion (normalized RGB ∈ [0,1]):**

Let $R, G, B \in [0, 1]$. First, compute:

- $C_{\text{max}} = \max(R, G, B)$
- $C_{\text{min}} = \min(R, G, B)$
- $\Delta = C_{\text{max}} - C_{\text{min}}$

Then compute HSV components:

$$
V = C_{\text{max}}
$$

$$
S = \begin{cases}
0 & \text{if } C_{\text{max}} = 0 \\
\frac{\Delta}{C_{\text{max}}} & \text{otherwise}
\end{cases}
$$

$$
H = \begin{cases}
0 & \text{if } \Delta = 0 \\
60^\circ \times \left( \frac{G - B}{\Delta} \mod 6 \right) & \text{if } C_{\text{max}} = R \\
60^\circ \times \left( \frac{B - R}{\Delta} + 2 \right) & \text{if } C_{\text{max}} = G \\
60^\circ \times \left( \frac{R - G}{\Delta} + 4 \right) & \text{if } C_{\text{max}} = B
\end{cases}
$$

**✍️ Note:**

- The OpenCV implementation of HSV scales:
  - **H to [0, 179]** instead of [0°, 360°]
  - **S and V to [0, 255]**


### <a id='toc4_5_1_'></a>[Using OpenCV](#toc0_)


In [ ]:
im_1_to_hsv = cv2.cvtColor(im_1, cv2.COLOR_RGB2HSV)

# plot
fig, axs = plt.subplots(1, 4, figsize=(16, 4), layout="compressed")
titles = ["HSV Image", "Hue Channel", "Saturation Channel", "Value Channel"]
images = [im_1_to_hsv] + [im_1_to_hsv[:, :, i] for i in range(3)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray" if img.ndim == 2 else None, vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

## <a id='toc4_6_'></a>[RGB - CMYK](#toc0_)

CMYK stands for **Cyan**, **Magenta**, **Yellow**, and **Key (Black)** — a subtractive color model used primarily in color printing.  
Unlike RGB (which is additive and designed for light-based displays), CMYK is based on ink or pigment absorption on physical media.

- **C (Cyan):** Absorbs red light.
- **M (Magenta):** Absorbs green light.
- **Y (Yellow):** Absorbs blue light.
- **K (Black):** Represents the black component used to enhance depth and reduce ink usage.

<div style="display: flex; justify-content: center; gap: 40px; align-items: flex-start; margin: 20px 0;">
  <div style="flex: 1; text-align: center; padding: 10px;">
    <img src="../../assets/images/third_party/RGB_combination_on_wall.png"
         alt="RGB_combination_on_wall.png"
         style="height: 500px; width: auto; object-fit: contain;">
    <div style="margin-top: 8px;">
      Additive Colors (©️
      <a href="https://en.wikipedia.org/wiki/Additive_color">Wikipedia - Additive color</a>)
    </div>
  </div>
  <div style="flex: 1; text-align: center; padding: 10px;">
    <img src="../../assets/images/third_party/SubtractiveColor.svg"
         alt="SubtractiveColor.svg"
         style="height: 500px; width: auto; object-fit: contain;">
    <div style="margin-top: 8px;">
      Subtractive Colors (©️
      <a href="https://en.wikipedia.org/wiki/Subtractive_color">Wikipedia - Subtractive color</a>)
    </div>
  </div>
</div>

**Purpose of RGB to CMYK Conversion:**

- **Printing:**  
  CMYK is the standard color model for printers and press workflows, as it corresponds directly to ink usage.

- **Color Separation:**  
  The black (K) channel allows better shadow reproduction and text clarity without combining all three color inks.

- **Cost Efficiency:**  
  Using K instead of combining C, M, and Y for dark tones reduces ink consumption and drying time.

🔢 **RGB to CMYK Conversion (normalized RGB ∈ [0,1]):**

Let $R, G, B \in [0, 1]$. First, compute the black component:

$$
K = 1 - \max(R, G, B)
$$

Then compute the CMY components:

$$
C = \frac{1 - R - K}{1 - K} \quad
M = \frac{1 - G - K}{1 - K} \quad
Y = \frac{1 - B - K}{1 - K}
$$

Special case:

- If \( R = G = B = 0 \) (pure black), then \( C = M = Y = 0 \), \( K = 1 \)

✍️ **Notes:**

- CMYK is **device-dependent**, meaning values may vary depending on printer profiles and paper type.
- RGB → CMYK is **not reversible** in general, because CMYK has a smaller color gamut.
- Normalized values are often scaled to \([0, 100]\%\) or \([0, 255]\) for practical implementations.


### <a id='toc4_6_1_'></a>[Using PIL](#toc0_)


In [ ]:
im_1_to_cmyk = np.array(Image.fromarray(im_1).convert('CMYK'))


# plot
fig, axs = plt.subplots(1, 5, figsize=(20, 4), layout="compressed")
titles = ["CMYK Image", "Cyan", "Magenta", "Yellow", "Key"]
images = [Image.fromarray(im_1).convert('CMYK')] + [im_1_to_cmyk[:, :, i] for i in range(4)]
for ax, img, title in zip(axs, images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=255)
    ax.set_title(title)
    ax.axis("off")
plt.show()

# <a id='toc5_'></a>[Visualize using Custom Color Maps](#toc0_)


## <a id='toc5_1_'></a>[Indexed Image](#toc0_)


In [ ]:
# fetch color palletes
colors = np.array(image_2.getpalette(), dtype=np.uint8).reshape(-1, 3)
num_colors =colors.shape[0]

# find alpha index
alpha_idx = np.ones(num_colors)
if 'transparency' in image_2.info:
    t = image_2.info['transparency']
    if isinstance(t, bytes):
        alpha_idx = np.array(list(t), dtype=np.uint8) / 255.0
    elif isinstance(t, int):
        alpha_idx[t] = 0.0

# add alpha channel + normalization
colors_norm = np.concatenate([colors / 255.0, alpha_idx[:, None]], axis=1)

# log
print(f"num colors : {num_colors}")
print(f"alpha idx  : {alpha_idx}")
print(f"colors:\n{colors_norm}")

In [ ]:
# create a color map
cmap = ListedColormap(colors_norm)

# plot
fig, axs = plt.subplots(1, 2, figsize=(8, 4), layout="compressed")
axs[0].imshow(im_2, cmap='gray', vmin=0, vmax=im_2.max())
axs[0].set_title("Wrong cmap")
axs[0].axis("off")
axs[1].imshow(im_2, cmap=cmap, vmin=0, vmax=num_colors)
axs[1].set_title("Correct cmap")
axs[1].axis("off")
plt.show()